In [2]:
#tes_data = "nerfstudio"
tes_data = "nerfmesh"

import trimesh
import open3d as o3d
import numpy as np
import pyembree
import tensorflow as tf

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2023-05-17 17:08:07.087415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 17:08:08.156969: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.set_visible_devices(gpus[0], 'GPU')

print("Test built: {}".format(tf.test.is_built_with_cuda()))

[]
Test built: True


2023-05-17 17:08:09.148204: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
# Load the ground truth and reconstructed meshes
ground_truth_file = '/home/nabil/WorkingFile/VE3DOR/ImageToStl.com_percobaan_kelima.obj'
if tes_data == "nerfstudio":
    reconstructed_file = '/home/nabil/WorkingFile/VE3DOR/nerfstudio_tsdf_mesh.ply'
elif tes_data == "nerfmesh":
    reconstructed_file = '/home/nabil/WorkingFile/VE3DOR/lego_v0.obj'
print(reconstructed_file)

/home/nabil/WorkingFile/VE3DOR/lego_v0.obj


In [8]:
def load_mesh(file_path):
    return trimesh.load_mesh(file_path)

def convert_trimesh_to_o3d(mesh):
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(mesh.vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(mesh.faces)
    return o3d_mesh

In [9]:
def compute_iou(mesh1, mesh2):
    mesh1_volume = mesh1.volume
    mesh2_volume = mesh2.volume
    #intersection_mesh = mesh1.intersection(mesh2)
    #intersection_mesh = mesh1.intersection(mesh2, engine='meshfix')
    intersection_mesh = mesh1.intersection(mesh2, engine='blender')
    intersection_volume = intersection_mesh.volume
    union_volume = mesh1_volume + mesh2_volume - intersection_volume
    return intersection_volume / union_volume

In [10]:
#def compute_chamfer_distance(pcd1, pcd2):
#    dist1 = o3d.pipelines.registration.compute_point_cloud_to_point_cloud_distance(pcd1, pcd2)
#    dist2 = o3d.pipelines.registration.compute_point_cloud_to_point_cloud_distance(pcd2, pcd1)
#    return np.mean(dist1) + np.mean(dist2)


def compute_chamfer_distance(pcd1, pcd2):
    dist1 = pcd1.compute_point_cloud_distance(pcd2)
    dist2 = pcd2.compute_point_cloud_distance(pcd1)
    return np.mean(dist1) + np.mean(dist2)


In [11]:
#def compute_f1_score(pcd1, pcd2, threshold):
#    dist1 = o3d.pipelines.registration.compute_point_cloud_to_point_cloud_distance(pcd1, pcd2)
#    dist2 = o3d.pipelines.registration.compute_point_cloud_to_point_cloud_distance(pcd2, pcd1)
#
#    precision = np.sum(np.array(dist1) < threshold) / len(dist1)
#    recall = np.sum(np.array(dist2) < threshold) / len(dist2)
#
#    f1_score = 2 * (precision * recall) / (precision + recall)
#    return f1_score

def compute_f1_score(pcd1, pcd2, threshold):
    dist1 = pcd1.compute_point_cloud_distance(pcd2)
    dist2 = pcd2.compute_point_cloud_distance(pcd1)

    precision = np.sum(np.array(dist1) < threshold) / len(dist1)
    recall = np.sum(np.array(dist2) < threshold) / len(dist2)

    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score


In [12]:
def visualize_meshes(meshes):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    for mesh in meshes:
        vis.add_geometry(mesh)

    vis.run()
    vis.destroy_window()

In [13]:
ground_truth_trimesh = load_mesh(ground_truth_file)
reconstructed_trimesh = load_mesh(reconstructed_file)

ground_truth_o3d = convert_trimesh_to_o3d(ground_truth_trimesh)
reconstructed_o3d = convert_trimesh_to_o3d(reconstructed_trimesh)

In [14]:
ground_truth_pcd = ground_truth_o3d.sample_points_uniformly(number_of_points=10000)
reconstructed_pcd = reconstructed_o3d.sample_points_uniformly(number_of_points=10000)

In [14]:
# Compute Chamfer Distance
chamfer_distance = compute_chamfer_distance(ground_truth_pcd, reconstructed_pcd)
print(f'Chamfer Distance: {chamfer_distance}')

Chamfer Distance: 0.0508463504107657


In [11]:
# Compute IoU
iou = compute_iou(ground_truth_trimesh, reconstructed_trimesh)
print(f'IoU: {iou}')

IoU: 0.17993334449135368


In [16]:
# Compute F1 Score
threshold = 0.01  # Define your own threshold based on the scale of your meshes
f1_score = compute_f1_score(ground_truth_pcd, reconstructed_pcd, threshold)
print(f'F1 Score: {f1_score}')

F1 Score: 0.054747762557077626


In [15]:
# def compute_hausdorff_distance(pcd1, pcd2):
#     return o3d.geometry.PointCloud.compute_hausdorff_distance(pcd1, pcd2)

from scipy.spatial.distance import directed_hausdorff

def compute_hausdorff_distance(pcd1, pcd2):
    # Convert Open3D.o3d.geometry.PointCloud to numpy array
    points1 = np.asarray(pcd1.points)
    points2 = np.asarray(pcd2.points)

    # Compute Hausdorff distance
    hausdorff_distance = max(directed_hausdorff(points1, points2)[0],
                             directed_hausdorff(points2, points1)[0])
    return hausdorff_distance

"""
In this code, directed_hausdorff computes the directed Hausdorff distance, which is not symmetric. 
To get the true Hausdorff distance, we compute it in both directions and take the maximum.
"""

# Compute Hausdorff Distance
hausdorff_distance = compute_hausdorff_distance(ground_truth_pcd, reconstructed_pcd)
print(f'Hausdorff Distance: {hausdorff_distance}')


Hausdorff Distance: 0.0997581855875862


In [ ]:
# # Visualize the meshes
visualize_meshes([ground_truth_o3d, reconstructed_o3d])

: 

: 